** This file gives a brief overview of the capabilities of the code. **

* The codes fit red arm spectra from Magellan/MIKE. 

* Since the codes are meant to be clean and minimal, it should not be hard to tweak the source codes for other purposes (say fitting the blue arm spectrum).

* The code fit for basic stellar parameters (Teff, logg, [Fe/H], [$\alpha$/Fe]), the broadening $v_{\rm broad}$, radial velocity, and continuum, to all spectral orders, simultaneously.

* Note that we does not assume any spectral mask here. Due to the imperfectness of Kurucz models, there will be non-negligible systematics. To mitigate that and impose your favorite spectral mask, simply set spectrum_err to large values (e.g., 999) to wavelength regions that you want mask out.

In [1]:
%matplotlib inline

# import packages
import numpy as np
from scipy.optimize import curve_fit
from scipy import interpolate
from scipy import signal
from scipy.stats import norm
import time 

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import gridspec

from cycler import cycler

# import The Payne (https://github.com/tingyuansen/The_Payne)
from Payne4MIKE import spectral_model
from Payne4MIKE import utils
from Payne4MIKE import fitting
from Payne4MIKE.read_spectrum import read_carpy_fits


In [2]:
# define plot properties
import matplotlib.cm as cm

from matplotlib import rcParams
from matplotlib import rc
from mpl_toolkits.axes_grid1 import make_axes_locatable

def rgb(r,g,b):
    return (float(r)/256.,float(g)/256.,float(b)/256.)

cb2 = [rgb(31,120,180), rgb(255,127,0), rgb(51,160,44), rgb(227,26,28), \
       rgb(10,10,10), rgb(253,191,111), rgb(178,223,138), rgb(251,154,153)]

rcParams['figure.figsize'] = (11,7.5)
rcParams['figure.dpi'] = 300

rcParams['lines.linewidth'] = 1

rcParams['axes.prop_cycle'] = cycler('color', cb2)
rcParams['axes.facecolor'] = 'white'
rcParams['axes.grid'] = False

rcParams['patch.facecolor'] = cb2[0]
rcParams['patch.edgecolor'] = 'white'

rcParams['font.family'] = 'Bitstream Vera Sans' 
rcParams['font.size'] = 25
rcParams['font.weight'] = 300


> Restore The Payne emulator.

In [3]:
# assuming Kurucz models
NN_coeffs, wavelength_payne = utils.read_in_neural_network()
w_array_0, w_array_1, w_array_2, b_array_0, b_array_1, b_array_2, x_min, x_max = NN_coeffs


> Read in MIKE spectra.

In [4]:
# an example of a MIKE spectrum to be fitted
wavelength, spectrum, spectrum_err = utils.read_in_example()

# or restore your own here
#read_path =  "star-109red_multi.fits"
#wavelength, spectrum, spectrum_err = read_carpy_fits(read_path)

#-----------------------------------------------------------------------------------
# restore a default hot star spectrum to determine telluric features
wavelength_blaze, spectrum_blaze, spectrum_err_blaze = utils.read_in_blaze_spectrum()

# or restore your own here
#read_path =  "Hot_Star_HR9087.fits"
#wavelength_blaze, spectrum_blaze, spectrum_err_blaze = read_carpy_fits(read_path)

#-----------------------------------------------------------------------------------
# match the order, some times reduction can drop some of the orders for low S/N data
dist = np.abs(wavelength[:, np.newaxis] - wavelength_blaze)
potentialClosest = dist.argmin(axis=1)[:,0]
wavelength_blaze = wavelength_blaze[potentialClosest,:]
spectrum_blaze = spectrum_blaze[potentialClosest,:]
spectrum_err_blaze = spectrum_err_blaze[potentialClosest,:]


/Users/yting/Payne4MIKE/Payne4MIKE/read_spectrum.py:98: RuntimeWarning: divide by zero encountered in power
  ivar = image[0].data[noise_ext]**(-2)


> Massaging the spectra into a digestable format.

In [5]:
# cull nonsensible values
spectrum = np.abs(spectrum)
spectrum_blaze = np.abs(spectrum_blaze)

# rescale the spectra by its median so it has a more reasonable y-range
spectrum, spectrum_err = utils.scale_spectrum_by_median(spectrum, spectrum_err)
spectrum_blaze, spectrum_err_blaze = utils.scale_spectrum_by_median(spectrum_blaze, spectrum_err_blaze)

# eliminate zero values in the blaze function to avoid dividing with zeros
# the truncation is quite aggresive, can be improved if needed
ind_valid = np.min(np.abs(spectrum_blaze), axis=0) != 0
spectrum_blaze = spectrum_blaze[:,ind_valid]
spectrum_err_blaze = spectrum_err_blaze[:,ind_valid]
wavelength_blaze = wavelength_blaze[:,ind_valid]

# match the wavelength (blaze -> spectrum)
spectrum_blaze, wavelength_blaze = utils.match_blaze_to_spectrum(wavelength, spectrum, wavelength_blaze, spectrum_blaze)

# use the blaze to determine telluric region
smooth_length = 30 # number of pixel in a block that we use to search for telluric features
threshold = 0.9
spectrum_err = utils.mask_telluric_region(spectrum_err, spectrum_blaze, smooth_length=30, threshold=0.9)


> Fit the spectrum.

In [6]:
# the range of RV that we will search (in the unit of 100 km/s)
# expand/refine the range of RV if the fit is stuck in a local minimum
RV_array=np.linspace(-2,2.,21)

# set boundaries for the fit [Teff [1000K], logg, Fe/H, Alpha/Fe, vbroad, RV [100 km/s]]
bounds = None

# teff_min, teff_max = x_min[0], x_max[0]
# logg_min, logg_max = x_min[1], x_max[1]
# feh_min, feh_max = x_min[2], x_max[2]
# alphafe_min, alphafe_max = x_min[3], x_max[3]
# vbroad_min, vbroad_max = 0.1, 10.
# RV_min, RV_max = -2., 2.

# bounds = np.zeros((2,6))
# bounds[0,0] = (teff_min - x_min[0])/(x_max[0]-x_min[0]) - 0.5
# bounds[1,0] = (teff_max - x_min[0])/(x_max[0]-x_min[0]) - 0.5
# bounds[0,1] = (logg_min - x_min[1])/(x_max[1]-x_min[1]) - 0.5
# bounds[1,1] = (logg_max - x_min[1])/(x_max[1]-x_min[1]) - 0.5
# bounds[0,2] = (feh_min - x_min[2])/(x_max[2]-x_min[2]) - 0.5
# bounds[1,2] = (feh_max - x_min[2])/(x_max[2]-x_min[2]) - 0.5
# bounds[0,3] = (alphafe_min - x_min[3])/(x_max[3]-x_min[3]) - 0.5
# bounds[1,3] = (alphafe_max - x_min[3])/(x_max[3]-x_min[3]) - 0.5
# bounds[0,-2] = vbroad_min
# bounds[1,-2] = vbroad_max
# bounds[0,-1] = RV_min
# bounds[1,-1] = RV_max

# perfort the fit
start_time = time.time()
popt_best, model_spec_best, chi_square = fitting.fit_global(spectrum, spectrum_err, spectrum_blaze, wavelength,\
                                                            NN_coeffs, wavelength_payne, RV_array=RV_array,\
                                                            polynomial_order=6, bounds_set=bounds)
print('Run Time : ', time.time()-start_time, ' s')

# save the results
np.savez("popt_best.npz",\
         popt_best=popt_best,\
         model_spec_best=model_spec_best,\
         chi_square=chi_square)

# print the best fit parameters
popt_best[:4] = (popt_best[:4] + 0.5)*(x_max-x_min) + x_min
popt_best[0] = popt_best[0]*1000.
print("[Teff [K], logg, Fe/H, Alpha/Fe] = ",\
      int(popt_best[0]*1.)/1.,\
      int(popt_best[1]*100.)/100.,\
      int(popt_best[2]*100.)/100.,\
      int(popt_best[3]*100.)/100.)
print("vbroad [km/s] = ", int(popt_best[-2]*10.)/10.)
print("RV [km/s] = ", int(popt_best[-1]*1000.)/10.)
print("Chi square = ", chi_square)


Pre Fit: Finding the best radial velocity initialization
1 / 21
2 / 21
3 / 21
4 / 21
5 / 21
6 / 21
7 / 21
8 / 21
9 / 21
10 / 21
11 / 21
12 / 21
13 / 21
14 / 21
15 / 21
16 / 21
17 / 21
18 / 21
19 / 21
20 / 21
21 / 21
Pre Fit: Fitting the blaze-normalized spectrum
1 / 1
Pre Fit: Finding the best continuum initialization
Final Fit: Fitting the whole spectrum with all parameters simultaneously
1 / 1
Run Time :  862.9925639629364  s
[Teff [K], logg, Fe/H, Alpha/Fe] =  4641.0 0.91 -0.51 -0.19
vbroad [km/s] =  2.0
RV [km/s] =  -39.7
Chi square =  11.492441417460423


> Plot the fits.

The telluric region is shaded in gray.

Blue is the observed spectrum, orange is the prediction +- the observation uncertainties.

In [ ]:
# make plot for individual order
for k in range(wavelength.shape[0]):
    fig = plt.figure(figsize=[18,20]);
    ax = fig.add_subplot(111)
    ax.spines['top'].set_color('none')
    ax.spines['bottom'].set_color('none')
    ax.spines['left'].set_color('none')
    ax.spines['right'].set_color('none')
    ax.tick_params(labelcolor='w', top='off', bottom='off', left='off', right='off')

#----------------------------------------------------------------------
    # zooming in the wavelength by plotting in a few panels
    for i in range(5):
    
        # wavelength range
        wavelength_min = np.min(wavelength[k,:])-10.
        wavelength_max = np.max(wavelength[k,:])+10.
        wave_period = (wavelength_max-wavelength_min)/5.
    
        # the yaxis range
        spec_min = np.min(spectrum[k,:])
        spec_max = np.max(spectrum[k,:])
        
        ax = fig.add_subplot(5,1,i+1)
        plt.xlim([wavelength_min+wave_period*(i),wavelength_min+wave_period*(i+1)])
        plt.ylim([spec_min-0.2,spec_max+0.2])
        
        # observe spectrum
        plt.plot(wavelength[k,:], spectrum[k,:], lw=2, label="MIKE", color=cb2[0])
        
        # best prediction
        plt.plot(wavelength[k,:], model_spec_best[k,:], label="Kurucz", lw=2, color=cb2[1])
        
        # plotting errors
        plt.fill_between(wavelength[k,:], model_spec_best[k,:]-spectrum_err[k,:],\
                         model_spec_best[k,:]+spectrum_err[k,:], alpha=0.5, color=cb2[1])
    
        # shade the telluric region in gray
        telluric_region = np.where(spectrum_err[k,:] == 999.)[0]
        start_telluric = np.where(np.diff(telluric_region) != 1)[0] ## find the blocks
        start_telluric = np.concatenate([[0], start_telluric+1, [telluric_region.size-1]])
        for m in range(start_telluric.size-1):
            telluric_block = wavelength[k,telluric_region[start_telluric[m]:start_telluric[m+1]]]
            num_telluric = telluric_block.size
            plt.fill_between(telluric_block, np.ones(num_telluric)*-10., np.ones(num_telluric)*10.,\
                             alpha=0.5, color="gray")
        
#----------------------------------------------------------------------
    # add axis and legend
    plt.xlabel("Wavelength [A]")
    plt.legend(loc="lower right", fontsize=28, frameon=False,\
                borderpad=0.05, labelspacing=0.1)

    # save figure
    plt.tight_layout()
    plt.savefig("Order_" +str(k+1) + ".png")
    plt.close()

findfont: Font family ['Bitstream Vera Sans'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Bitstream Vera Sans'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Bitstream Vera Sans'] not found. Falling back to DejaVu Sans.
